In [ ]:
# User based Recommendation (Collaborative filtering)
#if user A  and user B share similar profile then meal liked by User B but  not rated by A are recommended to A

In [5]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.neighbors import NearestNeighbors
df_profile = pd.read_csv('/Users/dipeshkumar/Desktop/final/Diet_Recommender/Data processing/user_Profiles.csv')
df_activity = pd.read_csv('/Users/dipeshkumar/Desktop/final/Diet_Recommender/Data processing/recent_activity.csv')
df_profile.head(3)

,User_Id,Veg_Non,Nutrient,Disease,Diet
0,User_1,non-veg,chloride,anemia kidney_disease goitre rickets pregnancy,type_a_diet high_protien_diet low_carb_diet l...
1,User_2,veg,chloride,goitre,high_fiber_diet vegan_diet
2,User_3,veg,magnesium,cancer hypertension goitre heart_disease scurvy,high_fiber_diet ketogenic_diet high_protien_diet


In [33]:
#Algorithm Used - K_Nearest-neighbor
class Recommender:
    
    def __init__(self):
        self.df = pd.read_csv('user_Profiles.csv')
        self.activity = pd.read_csv('user_activity.csv')
    
    def get_features(self):
        nutrient_dummies = self.df.Nutrient.str.get_dummies()
        disease_dummies = self.df.Disease.str.get_dummies(sep=' ')
        diet_dummies = self.df.Diet.str.get_dummies(sep=' ')
        feature_df = pd.concat([nutrient_dummies,disease_dummies,diet_dummies],axis=1)
        return feature_df
    
    def k_neighbor(self,inputs):
        feature_df = self.get_features()
        model = NearestNeighbors(n_neighbors=6,algorithm='ball_tree')
        model.fit(feature_df)
        
        df_results = pd.DataFrame(columns=list(self.df.columns))
        distnaces , indices = model.kneighbors(inputs)

        for i in list(indices):
            df_results = df_results.append(self.df.loc[i])
        df_results = df_results.reset_index(drop=True)
        return df_results
    
    def user_based(self,user_profile,user_id):
        data = self.get_features()
        total_features = data.columns
        d = dict()
        for i in total_features:
            d[i]= 0
        for i in user_profile:
            d[i] = 1
        final_input = list(d.values())
        
        similar_users = self.k_neighbor([final_input])
        users = list(similar_users.User_Id)
        results = self.activity[self.activity.User_Id.isin(users)] 
        results = results[results['User_Id']!=user_id]
        results = results.filter(['Name','Nutrient','Veg_Non','Price','Review','Diet','Disease','description'])

        results = results.drop_duplicates(subset=['Name'])
        results = results.reset_index(drop=True)
        return results

In [34]:
profile_A = ['calcium','iron','diabeties','gluten_free_diet']
user_id = 'User_71'

In [35]:
ob = Recommender()
data = ob.get_features()
total_features = data.columns
d = dict()
for i in total_features:
    d[i]= 0
for i in profile_A:
    d[i] = 1

final_input = list(d.values())
similar_users = ob.k_neighbor([final_input])
similar_users


,User_Id,Veg_Non,Nutrient,Disease,Diet
0,User_66,veg,protien,hypertension,gluten_free_diet
1,User_32,veg,iron,hypertension,high_fiber_diet vegan_diet gluten_free_diet l...
2,User_16,veg,iron,hypertension,high_fiber_diet vegan_diet gluten_free_diet l...
3,User_46,veg,protien,hypertension,dash_diet gluten_free_diet high_protien_diet
4,User_6,non-veg,iron,goitre,high_fiber_diet vegan_diet low_fat_diet
5,User_8,veg,vitamin_e,hypertension,dash_diet gluten_free_diet high_protien_diet


In [36]:
recommendations = ob.user_based(profile_A,user_id) 
recommendations

,Name,Nutrient,Veg_Non,Price,Review,Diet,Disease,description
0,almond and cranberry poha,iron,veg,425,9,high_fiber_diet vegan_diet gluten_free_diet l...,hypertension,"almond flakes, onion, poha, cranberries (froze..."
1,almond pearls,protien,veg,380,5,gluten_free_diet,hypertension,"toasted almonds, blueberries, oats, corn flake..."
2,oats khichdi,protien,veg,200,4,dash_diet gluten_free_diet high_protien_diet,hypertension,"क्विक कुकिंग ओट्स, मूंगदाल, जीरा, हल्दी पाउडर,..."
3,andhra pan fried pomfret,iron,veg,390,1,high_fiber_diet hormone_diet low_fat_diet veg...,cancer,"white pomfret fish, sunflower refined ooil, re..."
4,christmas tree pizza,chloride,veg,250,7,high_fiber_diet vegan_diet,goitre,"pizza dough (2 boules), red pepper, red onion,..."
5,chicken parmigiana with tomato sauce,vitamin_c,non-veg,475,10,hormone_diet low_carb_diet high_protien_diet ...,rickets goitre pregnancy kidney_disease anemia,"for chicken parmigiana:, chicken breast, egg w..."
6,chicken potli,iron,non-veg,340,10,high_fiber_diet hormone_diet low_carb_diet hi...,rickets goitre pregnancy kidney_disease anemia,"chicken, onion, green chilli, garlic, ginger, ..."
7,zucchini methi pulao,chloride,veg,270,7,ketogenic_diet,diabeties goitre,"zucchini, basmati rice, fenugreek (methi), cla..."
8,prawn fried rice,carbohydrates,veg,515,5,hormone_diet high_protien_diet dash_diet keto...,diabeties cancer obesity anemia,"तेल, बैकन, प्रॉन्स, चिकन, गाजर, लहसुन की कलिया..."
9,chocolate samosa,magnesium,veg,460,8,high_fiber_diet ketogenic_diet high_protien_diet,heart_disease hypertension,"refined flour, desi ghee, black cardamom seeds..."
